In [8]:
%pip install langchain openai faiss-cpu tiktoken pypdf langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [9]:
import os
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# Load medical documents
docs_path = "../docs"
documents = []

for file in os.listdir(docs_path):
    file_path = os.path.join(docs_path, file)

    # Load PDFs with PyPDFLoader
    if file.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())

    # Load text files with TextLoader
    elif file.endswith(".txt"):
        loader = TextLoader(file_path)
        documents.extend(loader.load())

# Split into smaller chunks for better retrieval
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

print(f"Loaded {len(chunks)} document chunks successfully!")


FileNotFoundError: [Errno 2] No such file or directory: '../docs'

In [13]:
!pip install -q sentence-transformers pymilvus


In [7]:
from pymilvus import MilvusClient, DataType

CLUSTER_ENDPOINT = "https://in03-22706440ab83439.serverless.gcp-us-west1.cloud.zilliz.com"
TOKEN = "fd3df03a845da1e42df2a2975f1d4733f62b044415d021f9f6047ef1f631504509c1a870e1370ebc8587c04883fae0c72c886204"

# 1. Set up a Milvus client
client = MilvusClient(
    uri=CLUSTER_ENDPOINT,
    token=TOKEN
)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Milvus
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType, connections, utility

# Define the collection name
collection_name = "document_embeddings"

# Connect to Milvus (ensure you've set up the connection correctly)
connections.connect("default", uri=CLUSTER_ENDPOINT, token=TOKEN)

# # List all collections in the connected Milvus instance
# existing_collections = utility.list_collections()

# # Check if the collection exists or create it
# if collection_name not in existing_collections:
#     # Define the schema and create the collection
#     embedding_dim = 768  # Example dimension, make sure this matches your model

#     fields = [
#         FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=embedding_dim),
#         FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
#     ]
#     schema = CollectionSchema(fields, description="Document embeddings for retrieval")

#     # Create the collection
#     collection = Collection(name=collection_name, schema=schema)
# else:
#     collection = Collection(collection_name)

# print(f"Collection '{collection_name}' is ready.")


In [ ]:
# from pymilvus import connections

# # Disconnect from all existing connections
# connections.disconnect("default")

# # Now try to connect again
# connections.connect("default", uri=CLUSTER_ENDPOINT, token=TOKEN)
# print("Successfully connected to Milvus!")


Successfully connected to Milvus!


In [9]:
# Load your local embeddings model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


<ipython-input-9-d3d5fa0a1e25>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
# Store the document embeddings into Milvus
try:
    vector_store = Milvus.from_documents(
        chunks,
        embeddings,
        collection_name=collection_name,
        connection_args={
            "uri": CLUSTER_ENDPOINT,
            "token": TOKEN,  # omit if unused
        }
    )
except Exception as e:
    print(f"❌ Failed to connect to Milvus cluster: {e}")

In [11]:
query = "What are the symptoms of asthma?"
retrieved_docs = vector_store.similarity_search(query, k=3)

print("Top Retrieved Documents:")
for doc in retrieved_docs:
    # print the source
    print(f"Source: {doc.metadata['source']}")
    # print the first 300 characters of the page content
    print(doc.page_content[:300])  # Print a snippet
    print("\n")


Top Retrieved Documents:
Source: ../docs\3.pdf
Home  Health  Conditions and Diseases
Signs of Respiratory
Distress
Signs of Respiratory Distress
Learning the signs of respiratory
distress
People having trouble breathing often show signs that they are having to work
harder to breathe or are not getting enough oxygen, indicating respiratory
dist


Source: ../docs\2.pdf
Global Regions
Donate
Home / Health topics / Chronic respiratory diseases
CreditsChronic respiratory diseases
Chronic respiratory diseases (CRDs) a ect the airways and other structures of the lungs. Some of the most
common are chronic obstructive pulmonary disease (COPD), asthma, occupational lung d


Source: ../docs\4.pdf
For Everyone 
Situation Summary
About Respiratory Illnesses
Risk Factors
Prevention
Treatment
Data and Trends
Resources to Prepare for Flu, COVID-19, and RSV
Respiratory Virus Guidance
Health Care Providers 
Best Practices for Patient Care
Talking with Patients
  Sign up for Email Updates  
ESPA


